In [ ]:
%load_ext autoreload
%autoreload 2

## Non-std libs
import pandas as pd
import requests

## Local modules
from scn_rrd.librenms_api_client import req_librenms
from scn_rrd import librenms_meta_utils

### Retrieve metadata about devices and ports, via LibreNMS's api.

Essentially we want to dump a portion of LibreNMS's mysql into pandas DataFrames.

In [ ]:
devices = req_librenms('/devices')
devices_df = pd.DataFrame(devices['devices'])

deviceGroups = req_librenms('/devicegroups')
devieGroups_df = pd.DataFrame(deviceGroups['groups'], columns=['id', 'name'])
devieGroups_df.rename(
    columns={'id': 'device_group_id', 'name': 'device_group_name'},
    inplace=True)

# To avoid this n+1 querying, we could `select * from device_group_device;`.
deviceGroups_devices = []
for deviceGrp in deviceGroups['groups']:
    deviceGrpId = deviceGrp['id']
    _devices = req_librenms(f"/devicegroups/{deviceGrpId}")
    _devices = _devices['devices']
    for _device in _devices:
        _device['device_group_id'] = deviceGrpId
    deviceGroups_devices.extend(_devices)
deviceGroups_devices_df = pd.DataFrame(deviceGroups_devices)

ports = req_librenms('/ports?columns=device_id,port_id')
ports_df = pd.DataFrame(ports['ports'])

portGroups = req_librenms('/port_groups')
portGroups_df = pd.DataFrame(portGroups['groups'])
portGroups_df.rename(
    columns={'id': 'port_group_id', 'name': 'port_group_name'},
    inplace=True)
portGroups_df.drop(columns='desc')

# To avoid this n+1 querying, we could `select * from librenms.port_group_port;`.
portGroups_ports = []
for portGrp in portGroups['groups']:
    portGrpId = portGrp['id']
    _ports = req_librenms(f"/port_groups/{portGrpId}")
    _ports = _ports['ports']
    for _port in _ports:
        _port['port_group_id'] = portGrpId
    portGroups_ports.extend(_ports)
portGroups_ports_df = pd.DataFrame(portGroups_ports)

### Extract relationships out of metadata.

In [ ]:
meta_df = (
    devices_df
    .merge(deviceGroups_devices_df, on='device_id', how='left')
    .merge(devieGroups_df, on='device_group_id', how='left')
    .merge(ports_df, on='device_id')
    .merge(portGroups_ports_df, on='port_id', how='left')
    .merge(portGroups_df, on='port_group_id', how='left')
)

In [ ]:
librenms_meta_utils.write_meta(meta_df)
# There is this error: RuntimeWarning: invalid value encountered in cast
# It's not clear where this is coming from. Is it the integer columns which become floats upon merge()ing?
# Data are intact, so we can ignore it, for now.

In [ ]:
librenms_meta_utils.read_meta()